In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import json
import open3d as o3d
import pandas as pd
import os
import copy
from utility import listFiles, quickVizNumpy
from scipy.spatial.transform import Rotation as R
from numpy import cos, sin, sqrt, arctan2, array
import torch.nn as nn
from einops import rearrange
### fucking around
# from pretrain import loadDataFromRegisteredSlam, cart2sph, preProcess
from train import getSpacing, getTargetCumulativeTransmittance, getTargetTerminationDistribution


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [62]:
path = r'/home/ansonhon/anson/thesis/LiDAR_NeRF/datasets/training_cumulative/building.npy'
data_cumulative = np.load(path)


In [63]:
ray_origin = data_cumulative[:,0:3]
ray_direction = data_cumulative[:,3:6]
ray_distance = data_cumulative[:,6]
ray_distance = rearrange(ray_distance, 'a -> a 1')

In [64]:
points = ray_direction * ray_distance + ray_origin


In [65]:
output = np.hstack((ray_origin, ray_direction, points))

In [67]:
quickVizNumpy(points[0:3000000,:])

In [68]:
np.save('building.npy', output)